In [9]:
import pandas as pd
import xgboost as xgb
import numpy as np
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier
from sklearn.metrics import (
    f1_score, 
    precision_score, 
    recall_score, 
    accuracy_score, 
    classification_report
)
from sklearn.model_selection import StratifiedKFold
import numpy as np
import pickle

In [10]:
data = pd.read_pickle("/Users/mac/Desktop/Code_Smell_Detection/dataset/feature_envy/codeT5_embeddings/processed_embeddings/codeT5_base_embed_line_by_line_sum.pkl")
data.head()

,emb_1,emb_2,emb_3,emb_4,emb_5,emb_6,emb_7,emb_8,emb_9,emb_10,...,emb_762,emb_763,emb_764,emb_765,emb_766,emb_767,emb_768,label,sample_id,severity
0,-7.521571,-7.762787,-10.407430,6.591759,7.070191,11.705015,2.640116,12.856184,-7.146132,-2.885672,...,3.887118,11.322876,-13.805436,8.518064,-5.794276,24.714706,-13.311764,1,12150,critical
1,-0.616862,-1.671327,-0.712171,1.246349,0.389413,1.486012,-0.136272,0.673206,0.153740,-0.755592,...,0.656833,0.333388,-0.605158,0.488275,-0.003898,1.431420,-0.495983,1,10168,critical
2,-11.226764,-15.170425,-18.570446,13.167607,4.495738,24.991566,6.812870,10.403657,1.928948,-12.261045,...,-2.955463,29.813223,-14.873217,15.325475,6.093493,35.506905,-19.922787,1,12783,critical
3,-7.613076,-22.431355,-27.734577,13.366014,4.421761,25.526434,3.585086,16.604767,0.640552,-9.345515,...,-7.696403,27.382660,-11.144313,17.118818,-2.077492,30.522797,-15.070001,1,12754,critical
4,-7.992052,-9.275466,-14.106435,11.998210,3.758430,11.559190,5.180464,7.495634,-1.092035,-8.209684,...,-0.886266,20.001431,-10.732095,13.953420,1.484560,26.704624,-11.948315,1,4277,critical


In [11]:
# Separate features and labels
X = data[[col for col in data.columns if col.startswith("emb_")]].values  # Select embedding columns
y = data['label'].values 

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Define classifiers
random_forest = RandomForestClassifier(n_estimators=100, random_state=42)
bagging = BaggingClassifier(estimator=RandomForestClassifier(), n_estimators=100, random_state=42)  
xgboost = xgb.XGBClassifier(n_estimators=100, random_state=42)

# Add the classifiers to a dictionary
classifiers = {
    "Random Forest": random_forest,
    "Bagging": bagging,
    "XGBoost": xgboost
}

In [12]:
# Initialize StratifiedKFold
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Dictionary to store cross-validation results
results = {}

# Perform cross-validation for each classifier
for clf_name, clf in classifiers.items():
    metrics_summary = {
        "f1_micro": [],
        "f1_macro": [],
        "accuracy": [],
        "precision_macro": [],
        "recall_macro": []
    }
    
    for train_idx, val_idx in skf.split(X_train, y_train):
        # Split the training data into training and validation folds
        X_train_fold, X_val_fold = X_train[train_idx], X_train[val_idx]
        y_train_fold, y_val_fold = y_train[train_idx], y_train[val_idx]
        
        # Train the classifier
        clf.fit(X_train_fold, y_train_fold)
        
        # Make predictions on the validation set
        y_pred = clf.predict(X_val_fold)
        
        # Calculate metrics
        metrics_summary["f1_micro"].append(f1_score(y_val_fold, y_pred, average='micro'))
        metrics_summary["f1_macro"].append(f1_score(y_val_fold, y_pred, average='macro'))
        metrics_summary["accuracy"].append(accuracy_score(y_val_fold, y_pred))
        metrics_summary["precision_macro"].append(precision_score(y_val_fold, y_pred, average='macro'))
        metrics_summary["recall_macro"].append(recall_score(y_val_fold, y_pred, average='macro'))
    
    # Store mean metrics for the classifier
    results[clf_name] = {metric: np.mean(scores) for metric, scores in metrics_summary.items()}

# Print cross-validation results
print("\nCross-Validation Results:")
for clf_name, metrics in results.items():
    print(f"\n{clf_name} Metrics:")
    for metric, value in metrics.items():
        print(f"{metric.capitalize()}: {value:.4f}")

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



Cross-Validation Results:

Random Forest Metrics:
F1_micro: 0.8593
F1_macro: 0.4873
Accuracy: 0.8593
Precision_macro: 0.5843
Recall_macro: 0.5090

Bagging Metrics:
F1_micro: 0.8631
F1_macro: 0.4795
Accuracy: 0.8631
Precision_macro: 0.5828
Recall_macro: 0.5068

XGBoost Metrics:
F1_micro: 0.8555
F1_macro: 0.5282
Accuracy: 0.8555
Precision_macro: 0.6237
Recall_macro: 0.5302


In [13]:
# Final evaluation on the test set
final_results = {}

for clf_name, clf in classifiers.items():
    # Train the classifier on the entire training set
    clf.fit(X_train, y_train)
    
    # Make predictions on the test set
    y_pred_test = clf.predict(X_test)
    
    # Calculate test metrics
    f1_micro_test = f1_score(y_test, y_pred_test, average='micro')
    f1_macro_test = f1_score(y_test, y_pred_test, average='macro')
    precision_test = precision_score(y_test, y_pred_test, average='macro')
    recall_test = recall_score(y_test, y_pred_test, average='macro')
    accuracy_test = accuracy_score(y_test, y_pred_test)
    report_test = classification_report(y_test, y_pred_test)
    
    # Save final metrics for the classifier
    final_results[clf_name] = {
        "f1_micro": f1_micro_test,
        "f1_macro": f1_macro_test,
        "precision_macro": precision_test,
        "recall_macro": recall_test,
        "accuracy": accuracy_test,
        "classification_report": report_test
    }

# Print final test results
print("\nFinal Test Results:")
for clf_name, metrics in final_results.items():
    print(f"\n{clf_name} Metrics:")
    for metric, value in metrics.items():
        if metric != "classification_report":
            print(f"{metric.capitalize()}: {value:.4f}")
    print(f"Classification Report:\n{metrics['classification_report']}")


Final Test Results:

Random Forest Metrics:
F1_micro: 0.8713
F1_macro: 0.5188
Precision_macro: 0.8731
Recall_macro: 0.5278
Accuracy: 0.8713
Classification Report:
              precision    recall  f1-score   support

           0       0.87      1.00      0.93       786
           1       0.88      0.06      0.11       123

    accuracy                           0.87       909
   macro avg       0.87      0.53      0.52       909
weighted avg       0.87      0.87      0.82       909


Bagging Metrics:
F1_micro: 0.8658
F1_macro: 0.4721
Precision_macro: 0.9328
Recall_macro: 0.5041
Accuracy: 0.8658
Classification Report:
              precision    recall  f1-score   support

           0       0.87      1.00      0.93       786
           1       1.00      0.01      0.02       123

    accuracy                           0.87       909
   macro avg       0.93      0.50      0.47       909
weighted avg       0.88      0.87      0.80       909


XGBoost Metrics:
F1_micro: 0.8614
F1_macro: 

In [15]:
# Save results to a pickle file
output_path = "/Users/mac/Desktop/Code_Smell_Detection/feature_envy/results/codeT5/base/result_codeT5_base_sum.pkl"
with open(output_path, "wb") as f:
    pickle.dump({"cross_val_results": results, "final_results": final_results}, f)
print(f"\nResults saved to {output_path}")


Results saved to /Users/mac/Desktop/Code_Smell_Detection/feature_envy/results/codeT5/base/result_codeT5_base_sum.pkl
